In [7]:
# !pip install -r requirements.txt

In [8]:
from src.constants import WINDOW_SIZE, PRETRAIN_EPOCHS
models = [
    "ENCDL", # The final solution, an encoder regressor with a window size of 19 and a learned positional encoding
    "ENCDS", # An encoder regressor  with a window size of 19 and a static positional encoding
    "LSTM", # An LSTM NN
    "RF", # A Random Forest model
    "LR", # A linear regressor,
    "ENCDRLfull", # The final solution but trained on all 4 files
]

model_paths = {
    "ENCDL" : f"saved_models/encdl-{WINDOW_SIZE}-{PRETRAIN_EPOCHS}.model",
    "ENCDS" : f"saved_models/encds-{WINDOW_SIZE}-{PRETRAIN_EPOCHS}.model",
    "LSTM" : "saved_models/lstm.keras",
    "RF" : "saved_models/rforest.model",
    "LR" : "saved_models/lregression.model",
    "ENCDRLfull" : f"saved_models/encd-full.model",
}
from src.ml import train_lr, train_rf, predict as predict_ml
from src.encoder import train as train_enc, predict as predict_enc
from src.lstm import train as train_lstm, predict as predict_lstm


model_train_functions = {
    "ENCDL" : lambda *args: train_enc(*args, True),
    "ENCDS" : lambda *args: train_enc(*args, False),
    "LSTM" : train_lstm,
    "RF" : train_rf,
    "LR" : train_lr,
    "ENCDRLfull" : lambda *args: train_enc(*args, True)

}

model_test_functions = {
    "ENCDL" : lambda *args: predict_enc(*args, True),
    "ENCDS" : lambda *args: predict_enc(*args, False),
    "LSTM" : predict_lstm,
    "RF" : predict_ml,
    "LR" : predict_ml,
    "ENCDRLfull" : lambda *args: predict_enc(*args, True)
}

model_result_paths =  {
    "ENCDL" : f"results_encdl-{WINDOW_SIZE}-{PRETRAIN_EPOCHS}.json",
    "ENCDS" : f"results_encds-{WINDOW_SIZE}-{PRETRAIN_EPOCHS}.json",
    "LSTM" : "results_lstm.json",
    "RF" : "results_rf.json",
    "LR" : "results_lr.json",
    "ENCDRLfull" : "results_full.json"
}

/!\ Chose here the model you want by changing the MODEL variable

In [9]:
MODEL = models[5]
PATH = model_paths[MODEL]
TRAIN_FUNC = model_train_functions[MODEL]
TEST_FUNC = model_test_functions[MODEL]
SAVE_PATH = model_result_paths[MODEL]
MODEL, WINDOW_SIZE, PRETRAIN_EPOCHS

('ENCDRLfull', 19, 3)

In [10]:
from src.preprocessing import scale, denoise, remove_features
from src.open_files import TRAIN_FILES

from src.preprocessing import windowing
from src.open_files import open_train_val
from src.constants import TRAIN_FILES, TEST_FILES, RUL_FILES

from src.postprocessing import align, aggregate2, aggregate
import numpy as np
np.random.seed(42)
import torch
torch.random.manual_seed(42)


Preprocessing

In [ ]:
df_train, df_val = open_train_val(TRAIN_FILES[0:1])
df_train = scale(denoise(remove_features(df_train)), True)
df_val = scale(denoise(remove_features(df_val)))
X_train, y_train, _tids_train, _cycles_train = windowing(df_train)
X_val, y_val, tids_val, cycles_val = windowing(df_val)

Training

In [ ]:
TRAIN_FUNC(X_train, y_train, X_val, y_val, PATH)
y_val_pred = TEST_FUNC(X_val, PATH)
df_pred = align(y_val_pred, cycles_val, tids_val)
val_ruls, y = aggregate2(df_pred, y_val)
print("Validation loss:", np.sqrt(np.mean((val_ruls - y)**2)))

testing

In [13]:
from src.open_files import open_test
df_test = open_test(TEST_FILES[0:1], RUL_FILES[0:1])
df_test = scale(denoise(remove_features(df_test)))
X_test, y_test, tids_test, cycles_test = windowing(df_test)
y_test_pred = TEST_FUNC(X_test, PATH)
df_pred = align(y_test_pred, cycles_test, tids_test)
test_ruls, y = aggregate(df_pred, y_test)

save results

In [14]:
import json
with open(SAVE_PATH, 'w') as f:
    json.dump(list(test_ruls), f)

with open(f'targets{WINDOW_SIZE}.json', 'w') as f:
    json.dump(list(y.astype(float)), f)

with open(SAVE_PATH.removesuffix('.json') + f'bywindow2.json', 'w')as f:
    print(SAVE_PATH.removesuffix('.json') + f'bywindow2.json')
    json.dump(
        {'pred' : list(y_test_pred.astype(float)),
         'tids' : list(tids_test),
         'cycles': list(cycles_test.astype(float)),
         'truth' : list(y_test.astype(float))
        }, f)

results_fullbywindow2.json


visualize results

In [15]:
import json

with open(SAVE_PATH, 'r') as f:
    test_ruls = np.array(json.load(f))

with open(f'targets{WINDOW_SIZE}.json', 'r') as f:
    y = np.array(json.load(f))

short_rul = y < 20
print(MODEL)
print("Overall Test loss:", np.sqrt(np.mean((test_ruls - y)**2)))
print("Test loss <20:", np.sqrt(np.mean((test_ruls - y)[short_rul]**2)))
print("Test loss >20:", np.sqrt(np.mean((test_ruls - y)[~short_rul]**2)))

ENCDRLfull
Overall Test loss: 424.02858231103204
Test loss <20: 195.50857135729765
Test loss >20: 448.2804571969074
